In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/Complaint data annotation (explain)_updated - cd (1).csv')

In [5]:
df.keys()

Index(['id', 'tweet', 'label', 'domain', 'sentiment', 'emotion', 'Severity',
       'Explain'],
      dtype='object')

In [6]:
df.head()

,id,tweet,label,domain,sentiment,emotion,Severity,Explain
0,1,@FC_HELP can I return online purchases to a Ho...,0,apparel,Neutral,other,0,can I return online purchases to a House of Fr...
1,2,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel,Positive,other,0,Hi - I'm writing a piece for MSN Him and wonde...
2,3,@FC_Help i need to check my order,0,apparel,Neutral,other,0,i need to check my order
3,4,@FC_Help I need to get in contact with someone...,1,apparel,Neutral,other,1,I need to get in contact with someone regardin...
4,5,@FC_Help How can I get a hold of you so we can...,0,apparel,Negative,other,0,How can I get a hold of you so we can discuss ...


In [7]:
domain_dict = {
    "other" : 0,
    "services" : 1,
    "random_reply" : 2,
    "software" : 3,
    "retail" : 4,
    "random_tweet" : 5,
    "transport" : 6,
    "cars" : 7,
    "food" : 8,
    "apparel" : 9,
    "electronics" : 10
}

# note to take string lower
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'joy' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

In [8]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
explain = []

for i in range(len(df)):
  if (pd.isna(df['label'][i]) or pd.isna(df['domain'][i]) or pd.isna(df['emotion'][i]) or pd.isna(df['sentiment'][i]) or pd.isna(df['Severity'][i]) or pd.isna(df['Explain'][i])):
    continue
  label.append(df['label'][i])
  domain.append(domain_dict[df['domain'][i]])
  emotion.append(emo_dict[df['emotion'][i]])
  sentiment.append(senti_dict[(df['sentiment'][i]).lower()])
  severity.append(df['Severity'][i])
  explain.append(df['Explain'][i])

In [9]:
from sklearn.model_selection import train_test_split

label_train, label_test, domain_train, domain_test, emotion_train, emotion_test, sentiment_train, sentiment_test, severity_train, severity_test, explain_train, explain_test = train_test_split(label, domain, emotion, sentiment, severity, explain, test_size = 0.2, random_state = 42, shuffle = True)
label_train, label_val, domain_train, domain_val, emotion_train, emotion_val, sentiment_train, sentiment_val, severity_train, severity_val, explain_train, explain_val = train_test_split(label_train, domain_train, emotion_train, sentiment_train, severity_train, explain_train, test_size = 0.2, random_state = 42, shuffle = True)

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(text):
  g_input_ids = []
  g_attention_mask = []

  for line in text:
    inputs = tokenizer(line, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60-size)), dim = 1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60-size)), dim = 1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    # Why we use np.array()
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))

  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [11]:
input_ids_train, attention_mask_train = preprocess_data(explain_train)
input_ids_val, attention_mask_val = preprocess_data(explain_val)
input_ids_test, attention_mask_test = preprocess_data(explain_test)

<ipython-input-10-eb1c066b164e>:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  g_input_ids = torch.tensor(g_input_ids)


In [12]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "domain" : torch.tensor(self.domain[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [13]:
from torch.utils.data import DataLoader

complaint_train = Complaint_dataset(input_ids_train, attention_mask_train, label_train, domain_train, emotion_train, sentiment_train, severity_train)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = Complaint_dataset(input_ids_val, attention_mask_val, label_val, domain_val, emotion_val, sentiment_val, severity_val)
val_dataloader = DataLoader(complaint_val, batch_size = 32, shuffle = False)

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [14]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

# What is super(Complaint) doing and why the name __init__() as such
# Practise OOPS

class Complaint(nn.Module):
  def __init__(self):
    super(Complaint, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased')
    self.complaint = nn.Linear(768, 2)
    self.severity = nn.Linear(768, 5)
    self.sentiment = nn.Linear(768, 3)
  def forward(self, input_ids, attention_mask):
    data = self.bert_model(input_ids = input_ids, attention_mask = attention_mask).pooler_output
    Complaint = self.complaint(data)
    Severity = self.severity(data)
    Sentiment = self.sentiment(data)
    return Complaint, Sentiment, Severity

In [15]:
complaint_model = Complaint()
complaint_model = complaint_model.to(device)
complaint_model.train()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(complaint_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

complaint_model.train()

for i in range(epochs):

  for data in train_dataloader:
    complaint_model.zero_grad()
    comp_train = data["label"].to(device)
    sev_train = data["severity"].to(device)
    emo_train = data["emotion"].to(device)
    senti_train = data["sentiment"].to(device)
    comp_out, senti_out, sev_out = complaint_model.forward(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

    loss_train = loss_func(comp_out, comp_train) + loss_func(sev_out, sev_train) + loss_func(senti_out, senti_train)
    loss_train.backward()
    optimizer.step()

    complaint_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        comp_val = data["label"].to(device)
        sev_val = data["severity"].to(device)
        emo_val = data["emotion"].to(device)
        senti_val = data["sentiment"].to(device)
        comp_val_out, senti_val_out, sev_val_out = complaint_model(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

        val_loss = loss_func(comp_val_out, comp_val) + loss_func(sev_val_out, sev_val) + loss_func(senti_val_out, senti_val)

        total_val = comp_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(complaint_model.state_dict(), "bert_model.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  complaint_model.load_state_dict(torch.load("bert_model.pt"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.cl

Epoch : 1
Validation loss : 6.128315418958664
Epoch : 2
Validation loss : 4.830819100141525
Epoch : 3
Validation loss : 4.611796081066132
Epoch : 4
Validation loss : 4.592790752649307
Epoch : 5
Validation loss : 4.644671812653542
Epoch : 6
Validation loss : 4.5782917737960815
Epoch : 7
Validation loss : 4.563957557082176
Epoch : 8
Validation loss : 4.576338678598404
Epoch : 9
Validation loss : 4.553625226020813
Epoch : 10
Validation loss : 4.584397941827774
Epoch : 11
Validation loss : 4.564986288547516
Epoch : 12
Validation loss : 4.5797237157821655
Epoch : 13
Validation loss : 4.560270965099335
Epoch : 14
Validation loss : 4.579674378037453
Epoch : 15
Validation loss : 4.568447604775429
Epoch : 16
Validation loss : 4.572821840643883
Epoch : 17
Validation loss : 4.560439839959145
Epoch : 18
Validation loss : 4.578538596630096
Epoch : 19
Validation loss : 4.564470291137695
Epoch : 20
Validation loss : 4.572267398238182


In [16]:
from sklearn.metrics import *

In [17]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 29.6 MB/s eta 0:00:00


In [18]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

senti_accuracy = MulticlassAccuracy(num_classes=3)
senti_precision = MulticlassPrecision(num_classes=3)
senti_f1 = MulticlassF1Score(num_classes=3)
senti_recall = MulticlassRecall(num_classes=3)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_senti = data["sentiment"]
      lab_sev = data["severity"]

      comp_out, senti_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_senti = torch.max(senti_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      senti_accuracy.update(pred_senti.cpu(), lab_senti.cpu())
      senti_precision.update(pred_senti.cpu(), lab_senti.cpu())
      senti_f1.update(pred_senti.cpu(), lab_senti.cpu())
      senti_recall.update(pred_senti.cpu(), lab_senti.cpu())
      senti_test_accuracy = senti_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute() 

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    senti_test_accuracy = senti_accuracy.compute()
    senti_test_precision = senti_precision.compute()
    senti_test_f1 = senti_f1.compute()
    senti_test_recall = senti_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

    print(f"senti_test_accuracy : {senti_test_accuracy}")
    print(f"senti_test_precision : {senti_test_precision}")
    print(f"senti_test_f1 : {senti_test_f1}")
    print(f"senti_test_recall : {senti_test_recall}")

<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-12-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


comp_test_accuracy : 0.8753623366355896
comp_test_precision : 0.8313252925872803
comp_test_f1 : 0.828000009059906
comp_test_recall : 0.824701189994812
sev_test_accuracy : 0.36296388506889343
sev_test_precision : 0.4484831988811493
sev_test_f1 : 0.3485836386680603
sev_test_recall : 0.36296388506889343
senti_test_accuracy : 0.6602089405059814
senti_test_precision : 0.6613126993179321
senti_test_f1 : 0.6606629490852356
senti_test_recall : 0.6602089405059814
